In [1]:
# inport BeautifulSoup
import re
from bs4 import BeautifulSoup

In [2]:
folder = "20170617"

In [3]:
file=open(folder + "/a.html", "r")
soup = BeautifulSoup(file.read(), "html.parser")
file.close()

In [4]:
column = 'Name,Age,TSI,PayPerWeek,Form,Stamina,Special,Experience,Leadership,Loyalty,Keeper,Defending,Playmaking,Winger,Passing,Scoring,SetPieces'
f = open(folder + '/player.txt', 'a', encoding='utf-8')
f.write(column +'\n')
f.close()

In [5]:
for div in soup.find_all("div","playerInfo"):
    # get String
    line = div.get_text()
    
    # parsing String
    line = line.replace(':\n                \n\n ', ':')
    line = line.replace('\n\n\n', '')
    line = line.replace('\n\n                        1', '\n1')
    line = line.replace('\n\n                        2', '\n2')
    line = line.replace('\n\n                        3', '\n3')
    line = line.replace('TSI = ','\nTSI:')
    line = line.replace('\xa0', '')
    line = line.replace(', 임금 = ', '\n임금:')
    line = line.replace('Won', ' Won\n')
    line = line.replace('form, ', 'form\n')
    if line.find('체력 [') > -1 :
        line = line.replace('체력 [', '체력\n특수능력:')
        line = line.replace("예측할 수 없음", "예측못함")
        line = line.replace(']', '\n')
    else :
        line = line.replace('체력', '체력\n특수능력:\n')
    line = line.replace('경험과 ', '경험\n')
    line = line.replace('리더십 능력', '리더십능력\n')
    line = line.replace('\n\n', '\n')
    line = line.replace('\n                    ', '')
    line = line.replace('충성도 ', '충성도:')
    line = line.replace('골키핑', '\n골키핑')
    line = line.replace('수비력','\n수비력')
    line = line.replace('플레이메이킹', '\n플레이메이킹')
    line = line.replace('윙어', '\n윙어')
    line = line.replace('패스력','\n패스력')
    line = line.replace('득점력', '\n득점력')
    line = line.replace('세트 피스','\n세트피스')
    
    line = line.replace('신의 경지','20')
    line = line.replace('초월적', '19')
    line = line.replace('마술적', '18')
    line = line.replace('전설적', '17')
    line = line.replace('외계인', '16')
    line = line.replace('막강함', '15')
    line = line.replace('초자연적', '14')
    line = line.replace('세계 최고', '13')
    line = line.replace('훌륭함', '12')
    line = line.replace('눈부심', '11')
    line = line.replace('걸출함', '10')
    line = line.replace('강력함', '9')
    line = line.replace('뛰어남', '8')
    line = line.replace('우수', '7')
    line = line.replace('무난함', '6')
    line = line.replace('불충분함', '5')
    line = line.replace('약함', '4')
    line = line.replace('나쁨', '3')
    line = line.replace('형편 없음', '2')
    line = line.replace('최악', '1')
    line = line.replace('없음', '0')
    
    line = line.replace('(\n', '(')
    line = line.replace('\n (', ':')
    line = line.replace(')', '')
    
    line = line.replace('Je-Min Jung    ', 'Je-Min Jung\n')
    line = line.replace('\n', ',')
    
    # name, age, TSI, pay per week, form, health(체력),
    
    line = line.replace(' 세', '세')
    line = line.replace(' 일', '일')
    line = line.replace('TSI:', '')
    line = line.replace('임금:주급 ', '')
    line = line.replace('0 Won','0')
    line = line.replace(' form', '')
    line = line.replace(' 체력', '')
    line = line.replace('특수능력:', '')
    line = line.replace(' 경험', '')
    line = line.replace(' 리더십능력', '')
    line = line.replace('충성도:', '')
    line = line.replace('골키핑:', '')
    line = line.replace('수비력:', '')
    line = line.replace('플레이메이킹:', '')
    line = line.replace('윙어:', '')
    line = line.replace('패스력:', '')
    line = line.replace('득점력:', '')
    line = line.replace('세트피스:', '')
    
    line = re.sub(r'[0-9]{4}-[0-9]{2}-[0-9]{2}:.*,', "", line)
    
    line.encode('utf-8')
    f = open(folder + '/player.txt', 'a', encoding='utf-8')
       
    for lines in line.split('\n'):
        if (re.match('^[0-9]{1,2}\. ', lines)) :
            lines = re.sub(r',$', "", lines)
            print(lines)
            f.write(lines +'\n')
    f.close()

1. Jee-Seung Oh,21세 61일,2670,990000,5,6,,3,4,20,6,3,1,1,1,1,6
2. Myung-Jin Min,1    28세 0일,1950,1090000,4,7,,5,1,20,6,4,1,1,2,1,2
3. Eun-Seok Hong,28세 95일,500,300000,5,6,,4,5,20,1,6,6,3,2,2,4
4. Ji-Woon Lee,18세 111일,910,390000,5,6,,2,5,20,1,7,4,3,5,2,5
5. Jang-Soo Choi,30세 45일,150,280000,3,6,빠름,5,5,20,1,6,4,4,2,1,7
6. Jae-Shin Cha,26세 92일,1140,390000,6,6,,4,5,20,1,7,3,6,3,2,6
7. In-Moo Byon,31세 80일,550,300000,5,6,,5,1,20,2,6,5,5,5,3,6
8. Hong-Chan Son,21세 40일,980,310000,6,6,,3,5,20,1,5,6,5,5,1,4
9. Hoi-Taek Park,19세 47일,770,330000,4,5,예측못함,2,2,20,1,5,6,3,6,3,6
10. Kwang-Eul Jeon,23세 46일,1370,430000,6,6,,3,4,20,1,6,7,3,3,2,5
11. Kwang-Hwan Yun,26세 13일,810,330000,6,6,,3,6,20,2,5,6,3,5,2,6
12. Chan-Myung Lee,25세 22일,970,410000,6,7,,4,3,20,1,3,7,3,4,3,4
13. Sung-Kook Han,21세 10일,1190,330000,5,6,공 마술사,3,2,20,1,5,6,6,5,2,5
14. Se-Jong Cho,28세 10일,710,290000,5,6,,4,4,20,1,5,5,6,3,4,4
15. Man-Ok Lee,22세 63일,1970,350000,6,6,예측못함,3,5,20,1,5,5,7,6,2,4
16. Dong-Shik Kim,18세 57일,990,410000,4,6,,2,6

In [ ]:
# cat player.txt

In [6]:
import csv

In [16]:
with open(folder + '/player.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match('^[0-9]{1,2}\. ', row[0])) :
            print(folder, row)
            insertPlayer(folder, row)

20170617 ['1. Jee-Seung Oh', '21세 61일', '2670', '990000', '5', '6', '', '3', '4', '20', '6', '3', '1', '1', '1', '1', '6']
20170617 ['2. Myung-Jin Min', '28세 0일', '1950', '1090000', '4', '7', '', '5', '1', '20', '6', '4', '1', '1', '2', '1', '2']
20170617 ['3. Eun-Seok Hong', '28세 95일', '500', '300000', '5', '6', '', '4', '5', '20', '1', '6', '6', '3', '2', '2', '4']
20170617 ['4. Ji-Woon Lee', '18세 111일', '910', '390000', '5', '6', '', '2', '5', '20', '1', '7', '4', '3', '5', '2', '5']
20170617 ['5. Jang-Soo Choi', '30세 45일', '150', '280000', '3', '6', '빠름', '5', '5', '20', '1', '6', '4', '4', '2', '1', '7']
20170617 ['6. Jae-Shin Cha', '26세 92일', '1140', '390000', '6', '6', '', '4', '5', '20', '1', '7', '3', '6', '3', '2', '6']
20170617 ['7. In-Moo Byon', '31세 80일', '550', '300000', '5', '6', '', '5', '1', '20', '2', '6', '5', '5', '5', '3', '6']
20170617 ['8. Hong-Chan Son', '21세 40일', '980', '310000', '6', '6', '', '3', '5', '20', '1', '5', '6', '5', '5', '1', '4']
20170617 ['9. Ho

In [10]:
import psycopg2

In [12]:
connect_str = "dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'"
connect_str

"dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'"

In [ ]:
try:
    # use our connection values to establish a connection
    connect_str = "dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'"
    conn = psycopg2.connect(connect_str)
    
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [ ]:
# create a psycopg2 cursor that can execute queries
cursor = conn.cursor()
cursor

In [ ]:
cursor.execute("""DROP TABLE IF EXISTS player """)
conn.commit()

In [ ]:
cursor.execute("""
CREATE TABLE player (
    Date DATE, 
    Name VARCHAR(30), 
    Age VARCHAR(10), 
    TSI BIGINT, 
    PayPerWeek BIGINT, 
    Form INT, 
    Stamina INT, 
    Special VARCHAR(8),
    Experience INT,
    Leadership INT,
    Loyalty INT,
    Keeper INT,
    Defending INT,
    Playmaking INT,
    Winger INT,
    Passing INT,
    Scoring INT,
    SetPieces INT
) ;""")
conn.commit()

In [ ]:
# insert a row to table
cursor.execute("""INSERT INTO player (SELECT to_date('20170609','YYYYMMDD')) ;""")
conn.commit()

In [15]:
def insertPlayer(folder, row):
    conn = None
    try:
        conn = psycopg2.connect(connect_str)
        cur = conn.cursor()
        sql = """INSERT INTO player (SELECT to_date(%s, 'YYYYMMDD'), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ;"""
        cur.execute(sql, (folder,row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16]))
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
# run a SELECT statement - no data in there, but we can try it
cursor.execute("""SELECT * from player ;""")
rows = cursor.fetchall()
print(rows)

In [13]:
def selectPlayer(tableName):
    conn = None
    try:
        conn = psycopg2.connect(connect_str)
        cur = conn.cursor()
        cur.execute("SELECT * FROM " + tableName)
        row = cur.fetchone()
        while row is not None:
            print(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
    finally:
        if conn is not None:
            conn.close()

In [17]:
selectPlayer("player")

(datetime.date(2017, 6, 10), '1. Jee-Seung Oh', '21세 59일', 2670, 990000, 5, 6, '', 3, 4, 20, 6, 3, 1, 1, 1, 1, 6)
(datetime.date(2017, 6, 10), '2. Myung-Jin Min', '27세 110일', 1930, 930000, 4, 7, '', 5, 1, 20, 6, 4, 1, 1, 2, 1, 2)
(datetime.date(2017, 6, 10), '3. Eun-Seok Hong', '28세 93일', 500, 300000, 5, 6, '', 4, 5, 20, 1, 6, 6, 3, 2, 2, 4)
(datetime.date(2017, 6, 10), '4. Ji-Woon Lee', '18세 109일', 910, 390000, 5, 6, '', 2, 5, 20, 1, 7, 4, 3, 5, 2, 5)
(datetime.date(2017, 6, 10), '5. Jang-Soo Choi', '30세 43일', 150, 280000, 3, 6, '빠름', 5, 5, 20, 1, 6, 4, 4, 2, 1, 7)
(datetime.date(2017, 6, 10), '6. Jae-Shin Cha', '26세 90일', 1120, 390000, 6, 6, '', 4, 5, 20, 1, 7, 3, 6, 3, 2, 6)
(datetime.date(2017, 6, 10), '7. In-Moo Byon', '31세 78일', 560, 300000, 5, 6, '', 5, 1, 20, 2, 6, 5, 5, 5, 3, 6)
(datetime.date(2017, 6, 10), '8. Hong-Chan Son', '21세 38일', 980, 310000, 6, 6, '', 3, 5, 20, 1, 5, 6, 5, 5, 1, 4)
(datetime.date(2017, 6, 10), '9. Hoi-Taek Park', '19세 45일', 770, 330000, 4, 5, '예측못함', 

In [ ]:
conn.close()